In [351]:
import csv
import numpy as np
import pandas as pd
import string

%matplotlib inline

# Functions

In [352]:
def format_table_output(raw, title = None):
    pre_text = '\\begin{table}\n\\small\n' 
    
    if title:
        pre_text = pre_text + '\\caption*{' + title + '}\n'
    
    post_text = '\\end{table}'
    return pre_text + raw + post_text

# Load Data

In [353]:
# Processed disclosure data
data_605 = pd.read_csv('../data/processed/605_processed.csv')
data_606 = pd.read_csv('../data/processed/606_processed.csv')

rawdata_605 = pd.read_csv('../data/605/rawdata_605.csv')
rawdata_606 = pd.read_csv('../data/606/rawdata_606.csv')

# Panel data
data_panel = pd.read_csv('../data/processed/regression_data_levels.csv')
data_panel.head()

,MarketCenter,Quarter,Exchange,OrderCode,CoveredOrders,CoveredShares,CancelledShares,MktCtrExecShares,AwayExecShares,ExecShares_0_9,...,Rel_PrImp_AvgT,MktCtrAvg_PrImp_ExpAmt,Rel_PrImp_ExpAmt,MktCtrAvg_All_AvgT,Rel_All_AvgT,Broker,Rebate,MktShare,Rebate_Dummy,Broker_Size
0,AQUA,2015Q4,Other,11,1,5000,5000,0,0,0,...,NaN,0.004768,NaN,20.208101,NaN,Deutsche,1.0,0.0,1,2.045009e+10
1,AQUA,2015Q4,Other,12,96,375200,223200,0,0,0,...,NaN,0.001124,NaN,19.623809,NaN,Deutsche,1.0,0.0,1,2.045009e+10
2,AQUA,2016Q1,Other,11,1,3000,3000,0,0,0,...,NaN,0.006184,NaN,21.218073,NaN,Deutsche,NaN,0.0,1,2.045009e+10
3,AQUA,2016Q1,Other,12,108,457600,202800,0,0,0,...,NaN,0.001188,NaN,15.650414,NaN,Deutsche,NaN,0.0,1,2.045009e+10
4,AQUA,2016Q2,NASDAQ,11,92,465340,158940,303400,0,296000,...,0.313173,0.010230,0.0196,0.200564,0.300425,Credit Suisse,1.0,0.0,1,3.266721e+10


# Summary Stats

## Market Centers

In [327]:
data_605_dsc = pd.DataFrame()
data_605['MarketCenter'].unique()
data_605 = data_605[data_605['MarketCenter'].apply(lambda x: x not in ['AQUA', 'DBAB', 'CITI'])].query('Quarter == "2017Q4"')
data_605.head()

,MarketCenter,Quarter,Exchange,OrderCode,CoveredOrders,CoveredShares,CancelledShares,MktCtrExecShares,AwayExecShares,ExecShares_0_9,...,PrImp_ExpAmt,All_AvgT,MktCtrAvg_PrImp_Pct,Rel_PrImp_Pct,MktCtrAvg_PrImp_AvgT,Rel_PrImp_AvgT,MktCtrAvg_PrImp_ExpAmt,Rel_PrImp_ExpAmt,MktCtrAvg_All_AvgT,Rel_All_AvgT
630,ARCA,2017Q4,NASDAQ,12,21623990,6020339726,3224272612,2705253712,88353686,2780305442,...,0.000622,2.860039,0.472153,-0.430638,0.554416,-0.540062,0.004107,-0.003484,15.498097,-12.638058
631,ARCA,2017Q4,NASDAQ,13,20410942,4478164854,4237870886,238529794,1091932,173875528,...,NaN,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,0.000000,0.000000
632,ARCA,2017Q4,NASDAQ,14,133189380,33779133270,32091616920,1677890954,8742880,815986972,...,NaN,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,0.000000,0.000000
633,ARCA,2017Q4,NASDAQ,15,212650020,64250659850,63818914142,428260994,3313580,133650822,...,NaN,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,0.000000,0.000000
634,ARCA,2017Q4,NYSE,12,30354840,8305057950,4041755052,4150815334,107844872,4235081538,...,0.000374,2.273444,0.480502,-0.447356,0.532010,-0.510225,0.002585,-0.002211,14.980760,-12.707316


### Dates

In [328]:
data_605_dsc['First_Quarter_obs'] = data_605.groupby(['MarketCenter']).first()['Quarter']
data_605_dsc['Last_Quarter_obs']  = data_605.groupby(['MarketCenter']).last()['Quarter']
data_605_dsc = data_605_dsc.reset_index()
data_605_dsc

,MarketCenter,First_Quarter_obs,Last_Quarter_obs
0,ARCA,2017Q4,2017Q4
1,BNYC,2017Q4,2017Q4
2,CDRG,2017Q4,2017Q4
3,EDGX,2017Q4,2017Q4
4,FBCO,2017Q4,2017Q4
5,G1ES,2017Q4,2017Q4
6,SGMA,2017Q4,2017Q4
7,UBSS,2017Q4,2017Q4
8,VRTU,2017Q4,2017Q4
9,WOLV,2017Q4,2017Q4


### Descriptive

In [329]:
descriptive_vars = ['MktCtrExecShares', 'AwayExecShares', 'PrImp_AvgAmt', 'PrImp_Pct', 'PrImp_ExpAmt', 'PrImp_AvgT', 'All_AvgT', 'AvgEffecSpread']

temp = data_605.groupby(['MarketCenter', 'Exchange', 'OrderType']).mean()[descriptive_vars].reset_index()
data_605_dsc = data_605_dsc.merge(temp)
data_605_dsc.head()

,MarketCenter,First_Quarter_obs,Last_Quarter_obs,Exchange,OrderType,MktCtrExecShares,AwayExecShares,PrImp_AvgAmt,PrImp_Pct,PrImp_ExpAmt,PrImp_AvgT,All_AvgT,AvgEffecSpread
0,ARCA,2017Q4,2017Q4,NASDAQ,Limit,2.705254e+09,8.835369e+07,0.014994,0.041516,0.000622,0.014354,2.860039,0.023519
1,ARCA,2017Q4,2017Q4,NASDAQ,Other,7.815606e+08,4.382797e+06,NaN,0.000000,NaN,NaN,0.000000,0.000000
2,ARCA,2017Q4,2017Q4,NYSE,Limit,4.150815e+09,1.078449e+08,0.011291,0.033146,0.000374,0.021785,2.273444,0.018340
3,ARCA,2017Q4,2017Q4,NYSE,Other,1.337434e+09,1.050750e+07,NaN,0.000000,NaN,NaN,0.000000,0.000000
4,ARCA,2017Q4,2017Q4,Other,Limit,3.364072e+09,7.126297e+07,0.006519,0.042627,0.000278,0.006279,2.799382,0.011750


In [330]:
panels = ['NASDAQ', 'NYSE', 'Other']
data_output = data_605_dsc.copy()

data_output['MktCtrExecShares'] = data_output['MktCtrExecShares'].apply(lambda x: np.int(x))
data_output['AwayExecShares'] = data_output['AwayExecShares'].apply(lambda x: np.int(x))
data_output['PrImp_Pct'] = data_output['PrImp_Pct'].apply(lambda x: str(np.round(x*100, decimals = 2)) + '%')

for i in range(0, len(panels)):
   
    data_table = data_output.query('Exchange == "' + panels[i] + '"').query('OrderType == "Market"')
    data_table = data_table[['MarketCenter'] + descriptive_vars]
    raw_latex_table = data_table.to_latex(index = False)
    
    table_header = 'Panel ' + string.ascii_uppercase[i] + ': ' + panels[i] + ' Stocks'
    
    print(format_table_output(raw_latex_table, title = table_header), end = '\n\n')

\begin{table}
\small
\caption*{Panel A: NASDAQ Stocks}
\begin{tabular}{lrrrlrrrr}
\toprule
MarketCenter &  MktCtrExecShares &  AwayExecShares &  PrImp\_AvgAmt & PrImp\_Pct &  PrImp\_ExpAmt &  PrImp\_AvgT &  All\_AvgT &  AvgEffecSpread \\
\midrule
        BNYC &         336059648 &               0 &      0.010616 &    76.88\% &      0.008162 &    0.344765 &  0.471176 &        0.021891 \\
        CDRG &        3187416192 &         4767260 &      0.011936 &     83.5\% &      0.009967 &    0.011501 &  0.023771 &        0.020737 \\
        EDGX &           2195098 &         1613214 &      0.023927 &    14.13\% &      0.003382 &    0.000111 &  0.002300 &        0.040493 \\
        FBCO &            646198 &        75410974 &      0.004836 &    69.35\% &      0.003354 &    0.168438 &  0.169756 &        0.051549 \\
        G1ES &        1357056996 &               0 &      0.013219 &    87.19\% &      0.011525 &    0.018875 &  0.036828 &        0.018940 \\
        SGMA &         832582914 &    

## Brokers

In [332]:
data_brokers_dsc = pd.DataFrame()

In [333]:
brokers = data_panel['Broker'].unique()

for broker in brokers:
    
    broker_rebate_dummy = data_panel.query('Broker == "' + broker + '"').iloc[0]['Rebate_Dummy']
    broker_connected_mktctrs = len(list(data_panel.query('Broker == "' + broker + '"')['MarketCenter'].unique()))
    
    data_brokers_dsc = data_brokers_dsc.append(
        {'Broker': broker, 'Rebate_Dummy': broker_rebate_dummy, 'MktCtr_Universe_Size': broker_connected_mktctrs}, 
        ignore_index=True)
    
    
data_brokers_dsc

,Broker,MktCtr_Universe_Size,Rebate_Dummy
0,Deutsche,10.0,1.0
1,Credit Suisse,10.0,1.0
2,Barclays Capital,6.0,1.0
3,BMO Capital,6.0,0.0
4,BTIG,7.0,1.0
5,TD Ameritrade,9.0,0.0
6,Insigneo Securities,7.0,0.0
7,Bank of the West,7.0,0.0
8,Boenning Scattergood,9.0,1.0
9,Edward Jones,8.0,0.0


In [334]:

data_606.iloc[l, 2]

NameError: name 'l' is not defined

In [335]:
mktctrs = list(data_605['MarketCenter'].unique())

data_606_mktshr = data_panel.groupby(['MarketCenter', 'Broker']).mean().reset_index()

data_606_mktshr = data_606_mktshr[data_606_mktshr['Broker'].apply(lambda x: x not in ['Securities America', 'Financorp Group', 'Cowen Execution'])]
data_606_mktshr = data_606_mktshr[data_606_mktshr['MarketCenter'].apply(lambda x: x not in ['AQUA', 'DBAB', 'CITI'] and x in mktctrs)]

display(data_606_mktshr.query('Broker == "AXA"'))

temp = (1-data_606_mktshr.groupby(['Broker']).sum()).reset_index()
temp['MarketCenter'] = 'zzOther'
data_606_mktshr = data_606_mktshr.append(temp)

,MarketCenter,Broker,OrderCode,CoveredOrders,CoveredShares,CancelledShares,MktCtrExecShares,AwayExecShares,ExecShares_0_9,ExecShares_10_29,...,MktCtrAvg_PrImp_AvgT,Rel_PrImp_AvgT,MktCtrAvg_PrImp_ExpAmt,Rel_PrImp_ExpAmt,MktCtrAvg_All_AvgT,Rel_All_AvgT,Rebate,MktShare,Rebate_Dummy,Broker_Size
15,CDRG,AXA,11.5,1.975408e+07,9.649342e+09,4.513703e+09,5.123999e+09,3.736775e+06,5.091627e+09,9.522803e+06,...,1.535822,-1.298185,0.020156,-0.015876,10.890043,-4.212620,0.0,0.182919,0.0,9999999.0
69,FBCO,AXA,11.5,2.210926e+05,1.614164e+08,3.124062e+06,1.518059e+08,6.267448e+06,1.548925e+08,1.617359e+06,...,5.686123,-4.182432,0.006725,-0.003847,18.555171,-1.146031,0.0,0.075254,0.0,9999999.0
82,G1ES,AXA,11.5,1.905134e+06,1.735751e+09,1.344666e+07,1.719789e+09,0.000000e+00,1.706505e+09,4.138305e+06,...,2.654761,-2.481326,0.017913,-0.013202,12.620060,-2.095333,0.0,0.195986,0.0,9999999.0
94,SGMA,AXA,11.5,3.651214e+06,2.064546e+09,4.904153e+08,1.572984e+09,0.000000e+00,1.564197e+09,2.323551e+06,...,0.339561,0.242785,0.036385,-0.031225,8.127136,-2.418531,0.0,0.068274,0.0,9999999.0
110,UBSS,AXA,11.5,2.066551e+07,8.636260e+09,6.282147e+09,1.472535e+09,5.607744e+08,2.009235e+09,7.436943e+06,...,1.702178,-1.329928,0.022210,-0.017533,11.525847,2.419196,0.0,0.092428,0.0,9999999.0
139,VRTU,AXA,11.5,1.118534e+07,7.102952e+09,3.279594e+09,3.817399e+09,0.000000e+00,3.758124e+09,3.450226e+07,...,0.521215,0.177063,0.004133,0.000911,8.203497,1.133635,0.0,0.115508,0.0,9999999.0


In [336]:
data_output = data_606_mktshr.copy()

temp = data_output.groupby('MarketCenter').mean().reset_index()
temp['Broker'] = 'zzAverage'
data_output = data_output.append(temp)

data_output['MktShare'] = data_output['MktShare'].fillna(0).apply(lambda x: np.round(x*100, decimals = 0))
data_output['MktShare'] = data_output['MktShare'].fillna(0).apply(lambda x: str(int(x)) + '%' if x > 0.00001 else '<1%')
data_output = data_output.pivot(columns = 'MarketCenter', values = 'MktShare', index = 'Broker')

display(data_output)
print(data_output.fillna('').to_latex().replace('<', '\\textless'))

MarketCenter,ARCA,BNYC,CDRG,EDGX,FBCO,G1ES,SGMA,UBSS,VRTU,WOLV,zzOther
Broker,,,,,,,,,,,
AXA,None,None,18%,None,8%,20%,7%,9%,12%,None,27%
Aurora Capital,None,None,44%,None,None,None,None,28%,None,None,28%
BGC Financial,None,None,None,None,None,None,None,None,<1%,None,100%
BMO Capital,4%,None,None,<1%,13%,None,None,<1%,<1%,<1%,83%
BTIG,<1%,None,1%,<1%,4%,None,None,<1%,1%,None,94%
Bank of the West,None,47%,7%,None,None,10%,4%,7%,10%,None,14%
Barclays Capital,7%,None,<1%,3%,<1%,None,<1%,<1%,None,None,89%
Benjamin Jerold,None,None,<1%,None,None,None,None,38%,None,None,62%
Boenning Scattergood,None,2%,22%,<1%,4%,5%,10%,13%,8%,None,36%


\begin{tabular}{llllllllllll}
\toprule
MarketCenter & ARCA & BNYC & CDRG & EDGX & FBCO & G1ES & SGMA & UBSS & VRTU & WOLV & zzOther \\
Broker                     &      &      &      &      &      &      &      &      &      &      &         \\
\midrule
AXA                        &      &      &  18\% &      &   8\% &  20\% &   7\% &   9\% &  12\% &      &     27\% \\
Aurora Capital             &      &      &  44\% &      &      &      &      &  28\% &      &      &     28\% \\
BGC Financial              &      &      &      &      &      &      &      &      &  \textless1\% &      &    100\% \\
BMO Capital                &   4\% &      &      &  \textless1\% &  13\% &      &      &  \textless1\% &  \textless1\% &  \textless1\% &     83\% \\
BTIG                       &  \textless1\% &      &   1\% &  \textless1\% &   4\% &      &      &  \textless1\% &   1\% &      &     94\% \\
Bank of the West           &      &  47\% &   7\% &      &      &  10\% &   4\% &   7\% &  10\% &      &  

In [337]:
data_606.query('Broker == "TD Ameritrade"')

,Broker,Exchange,OrderType,Quarter,Rebate,MarketCenter,MktShare,Rebate_Dummy,Obs_id
13060,TD Ameritrade,NASDAQ,Limit,2017Q1,0.0004,EDGX,0.1656,1,TD Ameritrade-EDGX-NASDAQ-Limit
13061,TD Ameritrade,NASDAQ,Limit,2017Q1,0.0011,VRTU,0.0814,1,TD Ameritrade-VRTU-NASDAQ-Limit
13062,TD Ameritrade,NASDAQ,Limit,2017Q1,0.0012,SGMA,0.0307,1,TD Ameritrade-SGMA-NASDAQ-Limit
13063,TD Ameritrade,NASDAQ,Limit,2017Q1,0.0013,G1ES,0.0483,1,TD Ameritrade-G1ES-NASDAQ-Limit
13064,TD Ameritrade,NASDAQ,Limit,2017Q1,0.0015,CDRG,0.4043,1,TD Ameritrade-CDRG-NASDAQ-Limit
13065,TD Ameritrade,NASDAQ,Limit,2017Q1,0.0029,CITI,0.1800,1,TD Ameritrade-CITI-NASDAQ-Limit
13066,TD Ameritrade,NASDAQ,Limit,2017Q3,0.0000,ARCA,0.0340,0,TD Ameritrade-ARCA-NASDAQ-Limit
13067,TD Ameritrade,NASDAQ,Limit,2017Q3,0.0004,EDGX,0.1292,1,TD Ameritrade-EDGX-NASDAQ-Limit
13068,TD Ameritrade,NASDAQ,Limit,2017Q3,0.0010,VRTU,0.1652,1,TD Ameritrade-VRTU-NASDAQ-Limit
13069,TD Ameritrade,NASDAQ,Limit,2017Q3,0.0011,SGMA,0.0372,1,TD Ameritrade-SGMA-NASDAQ-Limit


In [317]:
data_606_brk_info = data_606.groupby(['Broker']).mean().reset_index()#['Broker', 'Rebate_Dummy'].copy()
# data_606_brk_info['First_Quarter_obs'] = data_606.groupby(['Broker']).first()['Quarter']
# data_606_brk_info['Last_Quarter_obs']  = data_606.groupby(['Broker']).last()['Quarter']
# data_606_brk_info = data_606_brk_info.reset_index().drop('index', axis = 1)
data_606_brk_info

,Broker,Rebate,MktShare,Rebate_Dummy
0,AXA,0.000000,0.156312,0.000000
1,Aurora Capital,0.000000,0.302552,0.000000
2,BGC Financial,0.000000,0.123427,0.000000
3,BMO Capital,0.000000,0.070118,0.000000
4,BTIG,1.000000,0.058291,1.000000
5,Bank of the West,0.000000,0.130060,0.000000
6,Barclays Capital,1.000000,0.033506,1.000000
7,Benjamin Jerold,0.000000,0.298040,0.000000
8,Boenning Scattergood,1.000000,0.097962,1.000000
9,Buckingham Research,1.000000,0.128746,1.000000


In [357]:
data_panel.query('Rebate_Dummy == 1 & OrderType == "Market"')['Broker'].unique()

array(['Deutsche', 'Credit Suisse', 'Barclays Capital', 'BTIG',
       'TD Ameritrade', 'Boenning Scattergood', 'INTL FCStone',
       'JP Morgan', 'Evercore Group'], dtype=object)

In [233]:
data_606_mktshr.loc(:,'MktShare')

SyntaxError: invalid syntax (<ipython-input-233-3a90448a1ba1>, line 1)

In [270]:
temp = data_606.query('Broker == "Investment Placement Group"').copy()

brk = 'Investment Placement Group'

for mkt in temp['MarketCenter'].unique():
    
    temp2 = temp.query('MarketCenter == "' + mkt + '"')
    print(mkt)
    print(temp2['MktShare'].sum() / (3*3*len(data_606[data_606['Broker'] == brk]['Quarter'].unique())*len(data_606[data_606['Broker'] == brk]['MarketCenter'].unique())))

UBSS
0.005657866666666666
JPMS
0.015265377777777776
BNYC
0.04624697777777778
ATDF
0.0008146222222222222
CDRG
0.005379333333333333
KCG
0.0013392444444444444
SGMA
0.0027066666666666667
CITI
0.00011608888888888888
VRTU
0.005035111111111112
G1ES
0.0009939555555555555


In [227]:
3*3*len(data_606[data_606['Broker'] == "Investment Placement Group"]['Quarter'].unique())

225

In [271]:
len(data_606[data_606['Broker'] == brk]['MarketCenter'].unique())

10

In [273]:
3*3*len(data_606[data_606['Broker'] == brk]['Quarter'].unique())*len(data_606[data_606['Broker'] == brk]['MarketCenter'].unique())

2250

In [277]:
len(data_606.query('Broker == "Investment Placement Group"')['Quarter'].unique())

25

In [276]:
len(data_606.query('Broker == "Investment Placement Group"').query('MarketCenter == "CDRG"')['Quarter'].unique())

12

In [274]:
data_606.query('Broker == "Investment Placement Group"').query('MarketCenter == "CDRG"')

,Broker,Exchange,OrderType,Quarter,Rebate,MarketCenter,MktShare,Rebate_Dummy,Obs_id
19644,Investment Placement Group,NASDAQ,Limit,2015Q3,0.0,CDRG,0.0092,0,Investment Placement Group-CDRG-NASDAQ-Limit
19648,Investment Placement Group,NASDAQ,Limit,2015Q4,0.0,CDRG,0.1084,0,Investment Placement Group-CDRG-NASDAQ-Limit
19655,Investment Placement Group,NASDAQ,Limit,2016Q2,0.0,CDRG,0.2618,0,Investment Placement Group-CDRG-NASDAQ-Limit
19661,Investment Placement Group,NASDAQ,Limit,2016Q3,0.0,CDRG,0.2489,0,Investment Placement Group-CDRG-NASDAQ-Limit
19666,Investment Placement Group,NASDAQ,Limit,2016Q4,0.0,CDRG,0.1225,0,Investment Placement Group-CDRG-NASDAQ-Limit
19671,Investment Placement Group,NASDAQ,Limit,2017Q1,0.0,CDRG,0.1232,0,Investment Placement Group-CDRG-NASDAQ-Limit
19676,Investment Placement Group,NASDAQ,Limit,2017Q2,0.0,CDRG,0.1389,0,Investment Placement Group-CDRG-NASDAQ-Limit
19681,Investment Placement Group,NASDAQ,Limit,2017Q3,0.0,CDRG,0.1474,0,Investment Placement Group-CDRG-NASDAQ-Limit
19687,Investment Placement Group,NASDAQ,Limit,2017Q4,0.0,CDRG,0.1256,0,Investment Placement Group-CDRG-NASDAQ-Limit
19708,Investment Placement Group,NASDAQ,Market,2015Q3,0.0,CDRG,0.0746,0,Investment Placement Group-CDRG-NASDAQ-Market
